In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.applications.xception import Xception,preprocess_input
from tensorflow.keras.applications import DenseNet169,DenseNet201,MobileNetV2,ResNet50,VGG16,InceptionResNetV2,NASNetLarge
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
data = pd.read_csv("../input/whales/whales.csv")

In [4]:
data['species'].nunique()

30

In [5]:
data['species'].value_counts()

bottlenose_dolphin           9664
beluga                       7443
humpback_whale               7392
blue_whale                   4830
false_killer_whale           3326
dusky_dolphin                3139
spinner_dolphin              1700
melon_headed_whale           1689
minke_whale                  1608
killer_whale                 1493
fin_whale                    1324
gray_whale                   1123
bottlenose_dolpin            1117
kiler_whale                   962
southern_right_whale          866
spotted_dolphin               490
sei_whale                     428
short_finned_pilot_whale      367
common_dolphin                347
cuviers_beaked_whale          341
pilot_whale                   262
long_finned_pilot_whale       238
white_sided_dolphin           229
brydes_whale                  154
pantropic_spotted_dolphin     145
globis                        116
commersons_dolphin             90
pygmy_killer_whale             76
rough_toothed_dolphin          60
frasiers_dolph

In [6]:
data

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392
...,...,...,...
51028,fff639a7a78b3f.jpg,beluga,5ac053677ed1
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1184686361b3
51030,fff94675cc1aef.jpg,blue_whale,5401612696b9
51031,fffbc5dd642d8c.jpg,beluga,4000b3d7c24e


In [23]:
IMG_SIZE = 224
RESCALE = 1./255.

datagen = ImageDataGenerator(rescale=RESCALE,validation_split=0.2)

gen_kwargs = dict(
    dataframe=data,
    directory='../input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128', 
    x_col='image',
    y_col='species',
    batch_size=64,
    seed=42,
    shuffle=False,
    class_mode='categorical',
    target_size=(IMG_SIZE, IMG_SIZE)
)
train_generator = datagen.flow_from_dataframe(**gen_kwargs, subset="training")
valid_generator = datagen.flow_from_dataframe(**gen_kwargs, subset="validation")

Found 40827 validated image filenames belonging to 30 classes.
Found 10206 validated image filenames belonging to 30 classes.


In [24]:
from tensorflow.keras.applications import DenseNet201, VGG16, VGG19, ResNet50, InceptionV3, DenseNet201, Xception, MobileNetV2, EfficientNetB5

In [53]:
img_size = 224
base_model = DenseNet201(include_top = False,
                           weights = 'imagenet',
                           input_shape = (img_size,img_size,3))

for layer in base_model.layers[:]:
    layer.trainable = False

last_layer = base_model.get_layer('relu')
last_output = base_model.output

x = GlobalAveragePooling2D()(last_output)
x = Dense(30, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

In [54]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 230, 230, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
___________________________________________________________________________________________

In [55]:
model.compile(tf.keras.optimizers.RMSprop(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [56]:
model_path = "vgg16.h5"
checkpoint = ModelCheckpoint(model_path,
                         monitor="val_loss",
                         mode="min",
                         save_best_only = True,
                         verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                      min_delta = 0, 
                      patience = 5,
                      verbose = 1,
                      restore_best_weights = True)

callbacks_list = [checkpoint,earlystop]

vgg16_history = model.fit(
                train_generator,
                epochs=25,
                validation_data=valid_generator,
                callbacks=callbacks_list)

Epoch 1/25
638/638 [==============================] - 155s 229ms/step - loss: 0.8277 - accuracy: 0.7606 - val_loss: 0.5517 - val_accuracy: 0.8306

Epoch 00001: val_loss improved from inf to 0.55166, saving model to vgg16.h5
Epoch 2/25
638/638 [==============================] - 147s 230ms/step - loss: 0.4859 - accuracy: 0.8507 - val_loss: 0.4808 - val_accuracy: 0.8564

Epoch 00002: val_loss improved from 0.55166 to 0.48083, saving model to vgg16.h5
Epoch 3/25
638/638 [==============================] - 148s 232ms/step - loss: 0.4122 - accuracy: 0.8708 - val_loss: 0.4267 - val_accuracy: 0.8681

Epoch 00003: val_loss improved from 0.48083 to 0.42668, saving model to vgg16.h5
Epoch 4/25
638/638 [==============================] - 147s 230ms/step - loss: 0.3690 - accuracy: 0.8847 - val_loss: 0.4188 - val_accuracy: 0.8688

Epoch 00004: val_loss improved from 0.42668 to 0.41880, saving model to vgg16.h5
Epoch 5/25
638/638 [==============================] - 147s 231ms/step - loss: 0.3417 - accur

KeyboardInterrupt: 